In [10]:
#@@@@all topics addition commented to analyze difference with and without topic@@@@
import numpy as np
import pandas as p
import matplotlib.pyplot as plt
import cPickle
from sklearn.ensemble import GradientBoostingClassifier
documents_categories = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_categories.csv")
#clicks_test = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/clicks_test.csv", dtype={"display_id": int, "ad_id": int}))
#documents_meta = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_meta.csv")
#documents_entities = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_entities.csv")
promoted_content = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/promoted_content.csv", dtype = {"ad_id":int, "document_id":int, "campaign_id":int, "advertiser_id":int})
#sample_submission = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/sample_submission.csv")
documents_topics = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_topics.csv")
clicks_train = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/clicks_train.csv", dtype={"display_id": int, "ad_id": int, "clicked" : bool})
events = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/events.csv", dtype={"display_id": int, "uuid": str, "document_id" : int, "timestamp" : int, "platform" : str, "geo_location" : str})
#page_views = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/page_views.csv")
#page_views_sample = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/page_views_sample.csv")

In [11]:
#add document to clicks_train
events.drop(events.columns[[1,3,4,5]],axis = 1,inplace=True)
clicks_train = clicks_train.merge(events, how='left', on='display_id')
del events

In [ ]:
#1 - choose either 1 or 2
#split to validation and train, choosing display_id by random
display_ids = clicks_train.groupby(['display_id'])['display_id'].agg({'count' : 'count'}).reset_index().drop('count',axis = 1)
validation_displays = display_ids.sample(frac = 0.1)
clicks_train.reset_index(level=0, inplace=True)
validation = validation_displays.merge(clicks_train, how = 'inner', on = 'display_id')
train = clicks_train.drop(validation['index'])
del clicks_train

In [12]:
#2
#10% of full train for train, 10% validation
#split to validation and train, choosing display_id by random
display_ids = clicks_train.groupby(['display_id'])['display_id'].agg({'count' : 'count'}).reset_index().drop('count',axis = 1)
validation_displays = display_ids.sample(frac = 0.1)
display_ids_train = display_ids.drop(validation_displays.index)
train_displays = display_ids_train.sample(validation_displays.shape[0])
validation = validation_displays.merge(clicks_train, how = 'inner', on = 'display_id')
train = train_displays.merge(clicks_train, how = 'inner', on = 'display_id')
print train.shape
print validation.shape
del clicks_train, train_displays, validation_displays, display_ids

(8712129, 4)
(8714413, 4)


In [ ]:
#remove all confidence levels below 0.3, can decide on another criteria later on
documents_categories = documents_categories[documents_categories.confidence_level > 0.3]
documents_topics = documents_topics[documents_topics.confidence_level > 0.3]
#drop confidence level
documents_categories.drop('confidence_level',axis=1, inplace=True)
documents_topics.drop('confidence_level',axis=1, inplace=True)

In [ ]:
#prepare one-hot
topic_dummies = p.get_dummies(documents_topics['topic_id'],prefix="_ti")
categories_dummies = p.get_dummies(documents_categories['category_id'],prefix="_ci")
topics = documents_topics.drop('topic_id',1).join(topic_dummies)
topics = topics.groupby(['document_id']).sum().reset_index()
categories = documents_categories.drop('category_id',1).join(categories_dummies)
categories = categories.groupby(['document_id']).sum().reset_index()
del documents_categories, categories_dummies, topic_dummies , documents_topics

In [ ]:
print promoted_content.head()
print categories.head()
print topics.head()

In [ ]:
#create promoted, promoted content (ad information) with its' categories and topics
#remove advertiser_id and campaign id: promoted_content.drop(promoted_content.columns[[2,3]],axis = 1,inplace=True)
promoted = promoted_content.merge(categories, how='left', on='document_id')
promoted = promoted.merge(topics, how='left', on='document_id').drop('document_id', axis=1)
del promoted_content
#all Nulls are of documents with no topic or category, equals to 0
promoted.fillna(0.0, inplace=True)

In [37]:
#add ad_on_doc and ad_strength features
ad_on_doc = p.read_csv("/Users/astrachan/Desktop/outbrain/notebook/document_on_ad.csv")
ad_strength = p.read_csv("/Users/astrachan/Desktop/outbrain/notebook/ad_strength.csv")

In [39]:
#merge them into train and validation
train = train.merge(ad_strength, how = 'left', on = ['ad_id'])
train = train.merge(ad_on_doc, how = 'left', on = ['document_id', 'ad_id'], suffixes=('_str','_adoc'))
validation = validation.merge(ad_strength, how = 'left', on = ['ad_id'])
validation = validation.merge(ad_on_doc, how = 'left', on = ['document_id', 'ad_id'], suffixes=('_str','_adoc'))
del ad_on_doc, ad_strength

In [ ]:
#get predictors, dummy chunk just for label names
chunk = train[:10].merge(promoted,how = 'left',on = 'ad_id')
chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
predictors=[x for x in train.columns if x not in ['index','display_id','ad_id','document_id','clicked']]

In [ ]:
#not in chunks - build classifier
train = train.merge(categories, how = 'left', on = 'document_id')
train = train.merge(topics, how = 'left', on = 'document_id')
train = train.merge(promoted, how = 'left', on = 'ad_id')
alg = GradientBoostingClassifier(n_estimators = 500, max_depth=4, min_samples_split = 4, max_leaf_nodes = 4)
alg.fit(train[predictors], train["clicked"])

In [ ]:
#not in chunks - run on validation
validation = validation.merge(categories, how = 'left', on = 'document_id')
validation = validation.merge(topics, how = 'left', on = 'document_id')
validation = validation.merge(promoted, how = 'left', on = 'ad_id')
predY=[]
predY = list(alg.predict_proba(validation[predictors]).astype(float)[:,1])

In [ ]:
#chunks - number of iterations for training
print train.shape[0]/10**5

In [ ]:
#chunks
initiallize algorithm then fit it on chunks of size 10**5
i = 0
k = 2
alg = RandomForestClassifier(random_state=1, n_estimators=3, min_samples_split=4, min_samples_leaf=2, warm_start=True)
for chunk in np.array_split(train, train.shape[0]/10**5):
    alg.n_estimators += k
    chunk = chunk.merge(promoted,how = 'left',on = 'ad_id')
    chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk.fillna(0.0, inplace=True)
    alg.fit(chunk[predictors], chunk["clicked"])
    i += 1
    print i

In [ ]:
#save outputted tree with name [tree_name]
tree_name = 'fourthTree_n_est_drop_no_topics'
with open('/Users/astrachan/Desktop/outbrain/' + tree_name, 'wb') as f:
    cPickle.dump(alg, f)

In [ ]:
#chunks - number of iterations for validating
print (validation.shape[0]/10**5)

In [ ]:
#chunks - run outputted tree on chunks of size 10**5 of validation
predY=[]
i = 0
for chunk in np.array_split(validation[:10**6], validation[:10**6].shape[0]/10**5):
    chunk = chunk.merge(promoted,how = 'left',on = 'ad_id')
    chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk.fillna(0.0, inplace=True)
    chunk_pred=list(alg.predict_proba(chunk[predictors]).astype(float)[:,1])
    predY += chunk_pred
    i += 1
    print i
predY = list(alg.predict_proba(validation[predictors]).astype(float)[:,1])

In [37]:
#check score
predict = np.asarray(predY)
val_copy = validation.copy()
val_copy['predict'] = predict
#delete predict? validation? chunk? chunk_pred? predictors? predY? bentaim lo

In [38]:
#portion of correct display_ids predictions
max_rows = val_copy.groupby(['display_id'])['predict'].transform(max) == val_copy['predict']
final = val_copy[max_rows]
success = final[final['clicked'] == True]
float(len(success)) / float(len(final))

0.6920424300232756

In [39]:
#function evaluating how far were you from predicting right
val_copy.sort_values(['display_id', 'predict'], inplace=True, ascending=[True, False] )
val_copy["seq"] = np.arange(val_copy.shape[0])
Y_seq = val_copy[val_copy.clicked == 1].seq.values
Y_first = val_copy[['display_id', 'seq']].drop_duplicates(subset='display_id', keep='first').seq.values
Y_ranks = Y_seq - Y_first
score = np.mean(1.0 / (1.0 + Y_ranks))
print("MAP: %.12f" % score)

MAP: 0.816083156129
